# Employee and Bus Stops

## Context

A company XYZ intends to provide a bus shuttle service that would help its employees commute to the office. The company is based in Mountain View and the shuttle would provide transportation for employees based in San Francisco.

The city of San Francisco has given the company a list of potential bus stops that it may use. However, the company may use no more than 10 of these bus stops for its shuttle service.

The company XYZ is asking you to come up with the **10 most efficient bus stops** that would best serve its employees. Generally speaking, these "efficient" stops would result in the least walking distances between the employees' homes and their respective bus stops. To that end, you were given the following data:
- the list of bus stops provided by the city of San Francisco, `Bus_Stops.csv`
- a list of its employees' home addresses, `Employee_Addresses.csv`

Since trying out all possible combinations of 10 bus stops would take a prohibitively long time, the boss of XYZ has told you that you may simplify the problem and come up with 10 reasonable bus stops that are probably efficient.

## Objectives

- Explore and analyze the data. Provide comments on the outputs of your code and document your code well. 
- Feel free to show off your map visualization skills.
- Write an algorithm that produces the 10 best stops in your opinion. Also, please explain the rationale behind the algorithm. 
- Please the calculate the average walking distance per employee to their respective stops and report it at the end of your work.
- You may code the solution for this task in either Python or R. If you are coding in Python, you may enter your solution at the bottom of this notebook. Otherwise, you may create a new R jupyter notebook and copy this problem description over there. Either way, your solution is to be in the form of a jupyter notebook, regardless of the programming language used.
- Submit your work along with the data files used (`Bus_Stops.csv` & `Employee_Addresses.csv`) in a single ZIP file named as follows: `<FirstName>_<LastName>.zip`

## Evaluation

***Your solution will be evaluated on:***
- ***The soundness of the algorithm used to select the bus stops.***
- ***How much your code is neat, clear and well-documented.***
- ***Quality of narrative and commentary with interesting analyses and visuals.***

## Supplementary Notes

Prior to writing the requested algorithm, you will need to *geocode* the employees' home addresses and bus stops. You may use the [HERE REST APIs](https://developer.here.com/develop/rest-apis) for that purpose. Following are some links to help you in your task:
- To generate a free HereMaps account and an API Key to use for geocoding the addresses:  
    - https://developer.here.com/documentation/identity-access-management/dev_guide/topics/plat-using-apikeys.html
- Sections pertaining to *geocoding* in the documentation:  
    - https://developer.here.com/documentation/geocoder/dev_guide/topics/example-geocoding-free-form.html  
    - https://developer.here.com/documentation/geocoder/dev_guide/topics/example-geocoding-intersection.html
- Programmatically perform GET requests
    - Python: https://realpython.com/python-requests/
    - R: https://www.rdocumentation.org/packages/httr/versions/1.4.4

Note that HereMaps allows a maximum of 1000 requests per day, so it will take more than a single day to do all the geocoding. As a start, you may use all bus stops (~120 stops) and a few hundred employee addresses to start developing your algorithm. Save whatever you geocode so you would not need to geocode it again. Once the geocoding is done for all addresses, run the algorithm one last time and finalize your work.

*Side Note:*  
*The use of HereMaps API for geocoding is just one suggestion. If you are more comfortable using GoogleMaps API or OpenStreetMaps API, then you may use that as well. There are no constraints as to what you may use for geocoding of the addresses.*

<BR>
<BR>
<center><b><u>Finally, note that we will re-run your code (without the geocoding part) to make sure that your work is reproducible.</u></b></center>

<BR>
<center>
<H2>*** GOOD LUCK ***</H2>
</center>

--------

## Importing packages

In [29]:
import pandas as pd 
import numpy as np
import requests
import json

import plotly.graph_objects as go

## Read Data 

In [4]:
bus_stops = pd.read_csv("Bus_Stops.csv")
employee_addresses = pd.read_csv("Employee_Addresses.csv")

## Exploring Data 

In [5]:
print("Bus stops shape: ", bus_stops.shape)
bus_stops.head()

Bus stops shape:  (119, 2)


,Street_One,Street_Two
0,MISSION ST,ITALY AVE
1,MISSION ST,NEW MONTGOMERY ST
2,MISSION ST,01ST ST
3,MISSION ST,20TH ST
4,MISSION ST,FREMONT ST


In [6]:
print("Employee addresses shape: ", bus_stops.shape)
employee_addresses.head()

Employee addresses shape:  (119, 2)


,address,employee_id
0,"98 Edinburgh St, San Francisco, CA 94112, USA",206
1,"237 Accacia St, Daly City, CA 94014, USA",2081
2,"1835 Folsom St, San Francisco, CA 94103, USA",178
3,"170 Cambridge St, San Francisco, CA 94134, USA",50
4,"16 Roanoke St, San Francisco, CA 94131, USA",1863


In [7]:
bus_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Street_One  119 non-null    object
 1   Street_Two  119 non-null    object
dtypes: object(2)
memory usage: 2.0+ KB


In [8]:
employee_addresses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2191 entries, 0 to 2190
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   address      2191 non-null   object
 1   employee_id  2191 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 34.4+ KB


There are no missing data 

### Checking for duplicates 

In [9]:
bus_stops.duplicated(keep='last').tail()

114    False
115    False
116    False
117    False
118    False
dtype: bool

In [10]:
employee_addresses.duplicated(keep='last').tail()

2186    False
2187    False
2188    False
2189    False
2190    False
dtype: bool

There are no duplicates in the two dataframes 

### preparing the bus stops data for geocoding 

In [11]:
# check if there are values starts with 0, and remove the 0 -> to better geocoding  
for index, val in bus_stops['Street_Two'].iteritems():
    if val[0] == '0':
        bus_stops.at[index, 'Street_Two'] = val[1:]
        
bus_stops['address'] = bus_stops['Street_One'] + ', ' + bus_stops['Street_Two']

In [12]:
bus_stops.head()

,Street_One,Street_Two,address
0,MISSION ST,ITALY AVE,"MISSION ST, ITALY AVE"
1,MISSION ST,NEW MONTGOMERY ST,"MISSION ST, NEW MONTGOMERY ST"
2,MISSION ST,1ST ST,"MISSION ST, 1ST ST"
3,MISSION ST,20TH ST,"MISSION ST, 20TH ST"
4,MISSION ST,FREMONT ST,"MISSION ST, FREMONT ST"


## Geocoding 

In [13]:
def geocode_addresses(api_key, address_col, df, intersection):
    """ Geocode addresses 
    
    Arguments: 
        api_key (str): Here api key for geocoding 
        address_col (pandas series): it has the addresses 
        df (pandas dataframe): an empty data frame to store response, latitude and longitude
        intersection (bool): if we want to get the geocode for street intersection, set it to True
    
    Returns: 
        pandas dataframe that has the geocoded data 
    """
    
    url = "https://geocoder.ls.hereapi.com/6.2/geocode.json"
    
    for index, value in address_col.iteritems():
        address = value    # get address 
        print("{} {} is geocoded now ".format(index, address))
        
        # check if we want street intersection or a free-form address 
        if intersection:
            parameters = {
                'apiKey': api_key,
                'city': 'San Francisco',
                'street': address
            }
        else:
            parameters = {
                'apiKey': api_key,
                'searchtext': address
            }
            
        response = requests.get(url, params=parameters) 
        
        if (response.ok):            
            # convert the response from str to dict 
            res = json.loads(response.text)
            
            # get latitude and longitude values 
            lat = res['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']['Latitude']
            lon = res['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']['Longitude']
            
            # store data in the dataframe
            df.loc[len(df.index)] = [res, lat, lon]

        else:
            df.loc[len(df.index)] = [np.nan, np.nan, np.nan]
            
    return df 

### Geocode bus stops 

In [20]:
# creating new empty datafram to store response, latitude and longitude for each address 
bus_stops_gecoded = pd.DataFrame(columns=['response', 'latitude', 'longitude'])

In [21]:
bus_stops_gecoded.head()

,response,latitude,longitude


In [22]:
api_key = "Fw5VNn0p2uS6vbH90MI6q_1ocMhR6iN_X5ZVGxBGQDc"
bus_stops_gecoded = geocode_addresses(api_key, bus_stops['address'], bus_stops_gecoded, intersection=True)

0 MISSION ST, ITALY AVE is geocoded now 
1 MISSION ST, NEW MONTGOMERY ST is geocoded now 
2 MISSION ST, 1ST ST is geocoded now 
3 MISSION ST, 20TH ST is geocoded now 
4 MISSION ST, FREMONT ST is geocoded now 
5 MISSION ST, 13TH ST is geocoded now 
6 MISSION ST, ERIE ST is geocoded now 
7 MISSION ST, BEALE ST is geocoded now 
8 MISSION ST, FAIR AVE is geocoded now 
9 MISSION ST, SAINT MARYS AVE is geocoded now 
10 MISSION ST, SENECA AVE is geocoded now 
11 MISSION ST, ANTHONY ST is geocoded now 
12 MISSION ST, JESSIE EAST ST is geocoded now 
13 MISSION ST, SILVER AVE is geocoded now 
14 MISSION ST, MOUNT VERNON AVE is geocoded now 
15 MISSION ST, 30TH ST is geocoded now 
16 MISSION ST, MAIN ST is geocoded now 
17 MISSION ST, TRUMBULL ST is geocoded now 
18 MISSION ST, 15TH ST is geocoded now 
19 MISSION ST, KINGSTON ST is geocoded now 
20 MISSION ST, TINGLEY ST is geocoded now 
21 MISSION ST, THERESA ST is geocoded now 
22 MISSION ST, PARK ST is geocoded now 
23 MISSION ST, OLIVER ST is

In [23]:
bus_stops_gecoded.head()

,response,latitude,longitude
0,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.71675,-122.43594
1,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.78747,-122.40052
2,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.77867,-122.41160
3,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.75887,-122.41513
4,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.78987,-122.39595


In [24]:
bus_stops_gecoded.to_csv("bus_stops_gecoded.csv")

In [25]:
bus_stops = pd.concat([bus_stops, bus_stops_gecoded], axis=1)

In [26]:
bus_stops.head()

,Street_One,Street_Two,address,response,latitude,longitude
0,MISSION ST,ITALY AVE,"MISSION ST, ITALY AVE",{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.71675,-122.43594
1,MISSION ST,NEW MONTGOMERY ST,"MISSION ST, NEW MONTGOMERY ST",{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.78747,-122.40052
2,MISSION ST,1ST ST,"MISSION ST, 1ST ST",{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.77867,-122.41160
3,MISSION ST,20TH ST,"MISSION ST, 20TH ST",{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.75887,-122.41513
4,MISSION ST,FREMONT ST,"MISSION ST, FREMONT ST",{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.78987,-122.39595


### Geocode employee addresses 

In [14]:
employee_addresses_gecoded = pd.DataFrame(columns=['response', 'latitude', 'longitude'])

In [15]:
employee_addresses_gecoded

,response,latitude,longitude


In [16]:
employee_addresses['address']

0        98 Edinburgh St, San Francisco, CA 94112, USA
1             237 Accacia St, Daly City, CA 94014, USA
2         1835 Folsom St, San Francisco, CA 94103, USA
3       170 Cambridge St, San Francisco, CA 94134, USA
4          16 Roanoke St, San Francisco, CA 94131, USA
                             ...                      
2186       72 Topeka Ave, San Francisco, CA 94124, USA
2187           330 Oliver St, Daly City, CA 94014, USA
2188       94 W View Ave, San Francisco, CA 94134, USA
2189     1599 Geneva Ave, San Francisco, CA 94112, USA
2190       499 Treat Ave, San Francisco, CA 94110, USA
Name: address, Length: 2191, dtype: object

In [17]:
api_key = "Fw5VNn0p2uS6vbH90MI6q_1ocMhR6iN_X5ZVGxBGQDc"
employee_addresses_gecoded = geocode_addresses(api_key, employee_addresses['address'], employee_addresses_gecoded, intersection=False)

0 98 Edinburgh St, San Francisco, CA 94112, USA is geocoded now 
1 237 Accacia St, Daly City, CA 94014, USA is geocoded now 
2 1835 Folsom St, San Francisco, CA 94103, USA is geocoded now 
3 170 Cambridge St, San Francisco, CA 94134, USA is geocoded now 
4 16 Roanoke St, San Francisco, CA 94131, USA is geocoded now 
5 222 Whittier St, Daly City, CA 94014, USA is geocoded now 
6 13 Hyde St, San Francisco, CA 94102, USA is geocoded now 
7 2919 Geneva Ave, Daly City, CA 94014, USA is geocoded now 
8 100 Larkin St, San Francisco, CA 94102, USA is geocoded now 
9 230 Murray St, San Francisco, CA 94112, USA is geocoded now 
10 45 Sanchez St, San Francisco, CA 94114, USA is geocoded now 
11 2300 16th St, San Francisco, CA 94103, USA is geocoded now 
12 838 Schwerin St, Daly City, CA 94014, USA is geocoded now 
13 2111 Revere Ave, San Francisco, CA 94124, USA is geocoded now 
14 435 Vienna St, San Francisco, CA 94112, USA is geocoded now 
15 80 Risel Ave, Daly City, CA 94014, USA is geocoded n

128 925 Chenery St, San Francisco, CA 94131, USA is geocoded now 
129 318 Excelsior Ave, San Francisco, CA 94112, USA is geocoded now 
130 299 Melrose Ave, San Francisco, CA 94127, USA is geocoded now 
131 41 Britton St, San Francisco, CA 94134, USA is geocoded now 
132 233 Pope St, San Francisco, CA 94112, USA is geocoded now 
133 160 Bemis St, San Francisco, CA 94131, USA is geocoded now 
134 200 Cordova St, San Francisco, CA 94112, USA is geocoded now 
135 81 Tiffany Ave, San Francisco, CA 94110, USA is geocoded now 
136 45 Wilder St, San Francisco, CA 94131, USA is geocoded now 
137 3099 Washington St, San Francisco, CA 94115, USA is geocoded now 
138 50 Tiffany Ave, San Francisco, CA 94110, USA is geocoded now 
139 13th St, San Francisco, CA 94103, USA is geocoded now 
140 574 Campbell Ave, San Francisco, CA 94134, USA is geocoded now 
141 5601 Mission St, San Francisco, CA 94112, USA is geocoded now 
142 474 Naples St, San Francisco, CA 94112, USA is geocoded now 
143 1574 Howard

253 1399 15th St, San Francisco, CA 94103, USA is geocoded now 
254 134 Scotia Ave, San Francisco, CA 94124, USA is geocoded now 
255 737 Rolph St, San Francisco, CA 94112, USA is geocoded now 
256 2107 Lyon St, San Francisco, CA 94115, USA is geocoded now 
257 1399 15th St, San Francisco, CA 94103, USA is geocoded now 
258 531 Duboce Ave, San Francisco, CA 94117, USA is geocoded now 
259 135 Kissling St, San Francisco, CA 94103, USA is geocoded now 
260 2505 Green St, San Francisco, CA 94123, USA is geocoded now 
261 2101 Thomas Ave, San Francisco, CA 94124, USA is geocoded now 
262 2 Pearl St, San Francisco, CA 94103, USA is geocoded now 
263 1200 15th St, San Francisco, CA 94103, USA is geocoded now 
264 1390 Market St, San Francisco, CA 94102, USA is geocoded now 
265 3222 Washington St, San Francisco, CA 94115, USA is geocoded now 
266 345 Gambier St, San Francisco, CA 94134, USA is geocoded now 
267 151 Lippard Ave, San Francisco, CA 94131, USA is geocoded now 
268 899 Geneva Ave

378 361 Frankfort St, Daly City, CA 94014, USA is geocoded now 
379 399 Hampshire St, San Francisco, CA 94110, USA is geocoded now 
380 119 Utah St, San Francisco, CA 94103, USA is geocoded now 
381 360 Laidley St, San Francisco, CA 94131, USA is geocoded now 
382 1480 Folsom St, San Francisco, CA 94103, USA is geocoded now 
383 105 Sanchez St, San Francisco, CA 94114, USA is geocoded now 
384 102 Cordova St, San Francisco, CA 94112, USA is geocoded now 
385 68 Arnold Ave, San Francisco, CA 94110, USA is geocoded now 
386 70 Everson St, San Francisco, CA 94131, USA is geocoded now 
387 398 Silver Ave, San Francisco, CA 94112, USA is geocoded now 
388 1685 Bryant St, San Francisco, CA 94103, USA is geocoded now 
389 161 Boutwell St, San Francisco, CA 94124, USA is geocoded now 
390 976 Athens St, San Francisco, CA 94112, USA is geocoded now 
391 398 Silver Ave, San Francisco, CA 94112, USA is geocoded now 
392 283 Madrid St, San Francisco, CA 94112, USA is geocoded now 
393 299 Raymond 

502 808 Twin Peaks Blvd, San Francisco, CA 94131, USA is geocoded now 
503 808 Twin Peaks Blvd, San Francisco, CA 94131, USA is geocoded now 
504 4199 Mission St, San Francisco, CA 94112, USA is geocoded now 
505 2024 Revere Ave, San Francisco, CA 94124, USA is geocoded now 
506 620 Silver Ave, San Francisco, CA 94134, USA is geocoded now 
507 5715 Mission St, San Francisco, CA 94112, USA is geocoded now 
508 436 Murray St, San Francisco, CA 94110, USA is geocoded now 
509 3222 Washington St, San Francisco, CA 94115, USA is geocoded now 
510 599 S Van Ness Ave, San Francisco, CA 94110, USA is geocoded now 
511 30 Camellia Ave, San Francisco, CA 94112, USA is geocoded now 
512 1580 Geneva Ave, San Francisco, CA 94112, USA is geocoded now 
513 1544 Geneva Ave, San Francisco, CA 94112, USA is geocoded now 
514 151 Potrero Ave, San Francisco, CA 94103, USA is geocoded now 
515 1121 Athens St, San Francisco, CA 94112, USA is geocoded now 
516 618 Hanover St, Daly City, CA 94014, USA is geoc

...

1618 65 Rousseau St, San Francisco, CA 94112, USA is geocoded now 
1619 8 Topeka Ave, San Francisco, CA 94124, USA is geocoded now 
1620 Market Square, 1355 Market St, San Francisco, CA 94103, USA is geocoded now 
1621 554 Moscow St, San Francisco, CA 94112, USA is geocoded now 
1622 25 Whittier St, San Francisco, CA 94112, USA is geocoded now 
1623 1843 Felton St, San Francisco, CA 94134, USA is geocoded now 
1624 1231 Geneva Ave, San Francisco, CA 94112, USA is geocoded now 
1625 187 Edinburgh St, San Francisco, CA 94112, USA is geocoded now 
1626 298 Peru Ave, San Francisco, CA 94112, USA is geocoded now 
1627 38 Addison St, San Francisco, CA 94131, USA is geocoded now 
1628 5103 Mission St, San Francisco, CA 94112, USA is geocoded now 
1629 4083 Mission St, San Francisco, CA 94112, USA is geocoded now 
1630 128 Utah St, San Francisco, CA 94103, USA is geocoded now 
1631 941 Visitacion Ave, San Francisco, CA 94134, USA is geocoded now 
1632 100 Larkin St, San Francisco, CA 94102, US

1740 1201 Bryant St, San Francisco, CA 94103, USA is geocoded now 
1741 2345 Filbert St, San Francisco, CA 94123, USA is geocoded now 
1742 3118 Washington St, San Francisco, CA 94115, USA is geocoded now 
1743 491 Moscow St, San Francisco, CA 94112, USA is geocoded now 
1744 101, San Francisco, CA 94103, USA is geocoded now 
1745 264 Oriente St, Daly City, CA 94014, USA is geocoded now 
1746 199 Van Ness Ave, San Francisco, CA 94102, USA is geocoded now 
1747 474 Bellevue Ave, Daly City, CA 94014, USA is geocoded now 
1748 122 Rousseau St, San Francisco, CA 94112, USA is geocoded now 
1749 302 Potrero Ave, San Francisco, CA 94110, USA is geocoded now 
1750 715 Geneva Ave, San Francisco, CA 94112, USA is geocoded now 
1751 759 14th St, San Francisco, CA 94114, USA is geocoded now 
1752 98 Elliot St, San Francisco, CA 94134, USA is geocoded now 
1753 34 Walter St, San Francisco, CA 94114, USA is geocoded now 
1754 198 Amazon Ave, San Francisco, CA 94112, USA is geocoded now 
1755 5025 M

1864 150 Hampshire St, San Francisco, CA 94103, USA is geocoded now 
1865 899 Bayshore Blvd, San Francisco, CA 94124, USA is geocoded now 
1866 50 Alberta St, San Francisco, CA 94134, USA is geocoded now 
1867 394 Laidley St, San Francisco, CA 94131, USA is geocoded now 
1868 20 Mangels Ave, San Francisco, CA 94131, USA is geocoded now 
1869 600 Templeton Ave, Daly City, CA 94014, USA is geocoded now 
1870 1798 Visitacion Ave, San Francisco, CA 94134, USA is geocoded now 
1871 98 Everson St, San Francisco, CA 94131, USA is geocoded now 
1872 2116 Lyon St, San Francisco, CA 94115, USA is geocoded now 
1873 1699 Howard St, San Francisco, CA 94103, USA is geocoded now 
1874 238 Bellevue Ave, Daly City, CA 94014, USA is geocoded now 
1875 141 Hampshire St, San Francisco, CA 94103, USA is geocoded now 
1876 560 Vienna St, San Francisco, CA 94112, USA is geocoded now 
1877 699 Bayshore Blvd, San Francisco, CA 94124, USA is geocoded now 
1878 26 Athens St, San Francisco, CA 94112, USA is geoc

1988 310 Alemany Blvd, San Francisco, CA 94110, USA is geocoded now 
1989 450 10th St, San Francisco, CA 94103, USA is geocoded now 
1990 Twin Peaks Blvd, San Francisco, CA 94114, USA is geocoded now 
1991 11 Navajo Ave, San Francisco, CA 94112, USA is geocoded now 
1992 55 Trumbull St, San Francisco, CA 94112, USA is geocoded now 
1993 2300 16th St, San Francisco, CA 94103, USA is geocoded now 
1994 777 Twin Peaks Blvd, San Francisco, CA 94131, USA is geocoded now 
1995 531 Persia Ave, San Francisco, CA 94112, USA is geocoded now 
1996 35 Fell St, San Francisco, CA 94102, USA is geocoded now 
1997 30 Grove St, San Francisco, CA 94102, USA is geocoded now 
1998 221 Accacia St, Daly City, CA 94014, USA is geocoded now 
1999 3068 California St, San Francisco, CA 94115, USA is geocoded now 
2000 808 Twin Peaks Blvd, San Francisco, CA 94131, USA is geocoded now 
2001 737 Paris St, San Francisco, CA 94112, USA is geocoded now 
2002 681 Silver Ave, San Francisco, CA 94134, USA is geocoded no

2111 869 Schwerin St, Daly City, CA 94014, USA is geocoded now 
2112 625 Bayshore Blvd, San Francisco, CA 94124, USA is geocoded now 
2113 450 10th St, San Francisco, CA 94103, USA is geocoded now 
2114 338 Guerrero St, San Francisco, CA 94103, USA is geocoded now 
2115 59 Sheridan St, San Francisco, CA 94103, USA is geocoded now 
2116 562 Vienna St, San Francisco, CA 94112, USA is geocoded now 
2117 12 Shotwell St, San Francisco, CA 94103, USA is geocoded now 
2118 341 Bellevue Ave, Daly City, CA 94014, USA is geocoded now 
2119 186 Oriente St, Daly City, CA 94014, USA is geocoded now 
2120 1415 Folsom St, San Francisco, CA 94103, USA is geocoded now 
2121 2013 16th St, San Francisco, CA 94103, USA is geocoded now 
2122 2300 16th St, San Francisco, CA 94103, USA is geocoded now 
2123 530 Folsom St, San Francisco, CA 94105, USA is geocoded now 
2124 3065 Jackson St, San Francisco, CA 94115, USA is geocoded now 
2125 2300 16th St, San Francisco, CA 94103, USA is geocoded now 
2126 1844 

In [27]:
employee_addresses_gecoded.head()

,response,latitude,longitude
0,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.727642,-122.427315
1,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.704180,-122.415820
2,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.767930,-122.415100
3,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.729660,-122.419720
4,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.736240,-122.431320


In [19]:
employee_addresses_gecoded.to_csv("employee_addresses_gecoded.csv")

In [28]:
employee_addresses = pd.concat([employee_addresses, employee_addresses_gecoded], axis=1)
employee_addresses.head()

,address,employee_id,response,latitude,longitude
0,"98 Edinburgh St, San Francisco, CA 94112, USA",206,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.727642,-122.427315
1,"237 Accacia St, Daly City, CA 94014, USA",2081,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.704180,-122.415820
2,"1835 Folsom St, San Francisco, CA 94103, USA",178,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.767930,-122.415100
3,"170 Cambridge St, San Francisco, CA 94134, USA",50,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.729660,-122.419720
4,"16 Roanoke St, San Francisco, CA 94131, USA",1863,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.736240,-122.431320


### Save data as csv files and then read them again to run them easily from here 

In [30]:
employee_addresses.to_csv("employee_data.csv", index=False)
bus_stops.to_csv("bus_stops_data.csv", index=False)

In [31]:
employee_data  = pd.read_csv("employee_data.csv")
bus_stops_data = pd.read_csv("bus_stops_data.csv")

In [32]:
employee_data.head()

,address,employee_id,response,latitude,longitude
0,"98 Edinburgh St, San Francisco, CA 94112, USA",206,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.727642,-122.427315
1,"237 Accacia St, Daly City, CA 94014, USA",2081,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.704180,-122.415820
2,"1835 Folsom St, San Francisco, CA 94103, USA",178,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.767930,-122.415100
3,"170 Cambridge St, San Francisco, CA 94134, USA",50,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.729660,-122.419720
4,"16 Roanoke St, San Francisco, CA 94131, USA",1863,{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.736240,-122.431320


In [106]:
employee_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2191 entries, 0 to 2190
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   address      2191 non-null   object 
 1   employee_id  2191 non-null   int64  
 2   response     2191 non-null   object 
 3   latitude     2191 non-null   float64
 4   longitude    2191 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 85.7+ KB


In [107]:
employee_data.describe()

,employee_id,latitude,longitude
count,2191.000000,2191.000000,2191.000000
mean,1096.000000,37.744349,-122.425281
std,632.631541,0.026933,0.013984
min,1.000000,37.702862,-122.451970
25%,548.500000,37.721080,-122.435535
50%,1096.000000,37.736104,-122.425570
75%,1643.500000,37.767920,-122.414620
max,2191.000000,37.798920,-122.389980


In [33]:
bus_stops_data.head()

,Street_One,Street_Two,address,response,latitude,longitude
0,MISSION ST,ITALY AVE,"MISSION ST, ITALY AVE",{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.71675,-122.43594
1,MISSION ST,NEW MONTGOMERY ST,"MISSION ST, NEW MONTGOMERY ST",{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.78747,-122.40052
2,MISSION ST,1ST ST,"MISSION ST, 1ST ST",{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.77867,-122.41160
3,MISSION ST,20TH ST,"MISSION ST, 20TH ST",{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.75887,-122.41513
4,MISSION ST,FREMONT ST,"MISSION ST, FREMONT ST",{'Response': {'MetaInfo': {'Timestamp': '2022-...,37.78987,-122.39595


In [108]:
bus_stops_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Street_One  119 non-null    object 
 1   Street_Two  119 non-null    object 
 2   address     119 non-null    object 
 3   response    119 non-null    object 
 4   latitude    119 non-null    float64
 5   longitude   119 non-null    float64
dtypes: float64(2), object(4)
memory usage: 5.7+ KB


In [109]:
bus_stops_data.describe()

,latitude,longitude
count,119.000000,119.000000
mean,37.747559,-122.423357
std,0.026985,0.016045
min,37.708540,-122.452550
25%,37.723355,-122.436085
50%,37.743990,-122.420060
75%,37.772950,-122.412810
max,37.793550,-122.389460


### Plotting employee addresses and bus stops 

In [105]:
mapbox_access_token = open(".mapbox_token").read()

fig = go.Figure()

fig.add_trace(go.Scattermapbox(
        lat=employee_data['latitude'],
        lon=employee_data['longitude'],
        mode='markers',
        name="Employee Addresses",
        marker=go.scattermapbox.Marker(
            size=10,
            color='rgb(0, 0, 255)',
        ),
        text=employee_data['employee_id'],
        hoverinfo='text'
    ))

fig.add_trace(go.Scattermapbox(
        lat=bus_stops_data['latitude'],
        lon=bus_stops_data['longitude'],
        mode='markers',
        name="Bus Stops",
        marker=go.scattermapbox.Marker(
            size=10,
            color='rgb(0, 255, 0)',
        ),
        text=bus_stops_data['Street_Two'],
        hoverinfo='text',
    ))

fig.update_layout(
    title='Employee addresses and bus stops locations',
    autosize=True,
    hovermode='closest',
    showlegend=True,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=37.77,
            lon=-122.5
        ),
        pitch=0,
        zoom=10,
        style='light'
    ),
)

fig.show()